In [4]:
import requests
import json
import pandas as pd
import tempfile as tf
import uuid

In [2]:
baseurl = 'https://ccd1.vm.bridges.psc.edu/ccd-api'

# change the Basic authorization for your username/password (you can use Postman to generate) 
basicheader = {
    'authorization': "Basic <Postman>",    # CHANGE ME:  
    'accept': "application/json"
    }

# endpoints
jwturl = baseurl + '/jwt'
dataurl = baseurl + '/data'

In [5]:
# get JSON Web token and userId
r = requests.get(jwturl, headers=basicheader)
jwtresults = json.loads(r.text)

userId = jwtresults['userId']
jwt = jwtresults['jwt']

print (jwtresults)

# construct a header with the JSON web token
jtoken = "Bearer " + jwt
bearheader = {
    'authorization': jtoken,
    'accept': "application/json"    
    }
print (bearheader)

KeyError: 'userId'

In [6]:
fo = open('F:/GraduateStudy/2016Fall/DatAna/Assignment/7/train.csv', 'rb')
files = {'file': fo }

In [7]:
# post the newly saved data file from PIC-SURE query to your user directory in CCD
url = baseurl + '/' + str(userId) + '/dataset/upload'
r = requests.post(url, headers=bearheader, files=files)
print (r.text)

NameError: name 'userId' is not defined

In [8]:
fo.close()
print (json.dumps(json.loads(r.text), indent=4))  # pretty print the results
p_resultId = json.loads(r.text)  #  parse and get the job id
fileId = p_resultId['id']

{
    "status": 404, 
    "timestamp": 1480268289081, 
    "message": "HTTP 404 Not Found", 
    "path": "/error", 
    "error": "Not Found"
}


KeyError: 'id'

In [ ]:
payload = "{\n    \"id\": " + str(fileId) + ",\n    \"variableType\": \"continuous\",\n    \"fileDelimiter\": \"tab\"\n}"
bearheader.update({'content-type': "application/json"})    

In [ ]:
url = baseurl + '/' + str(userId) + '/dataset/summarize'
# post a request to summarize the data file
r = requests.post(url, data=payload, headers=bearheader)
print (r.status_code)
print (r.text)

In [ ]:
payload = "{\n    \"datasetFileId\": " + str(fileId)+ ",\n    \"dataValidation\": {\n      \"uniqueVarName\": true,\n      \"limitNumOfCategory\": false\n    },\n    \"algorithmParameters\": {\n      \"maxDegree\": -1,\n      \"penaltyDiscount\": 2},\n    \"jvmOptions\": {\n      \"maxHeapSize\": 10\n    }\n}"

In [ ]:
# use the alogrithm name as endpoint
# url = baseurl + '/' + str(userId) + '/jobs/gfcic'  # GFCI continuous
# url = baseurl + '/' + str(userId) + '/jobs/fgsd'  # FGS discrete
url = baseurl + '/' + str(userId) + '/jobs/fgsc'   # FGS Continuous
print (url)
r = requests.post(url, data=payload, headers=bearheader)
print (r.text)
p = json.loads(r.text)

In [ ]:
#  parse and get the job id, and expected result and error name 
jobId = p['id']
resultFilename = p['resultFileName']
errorFilename = p['errorResultFileName']
print (r.text)

In [ ]:
url = baseurl + '/' + str(userId) + '/jobs'
r = requests.get(url + '/' + str(jobId), headers=bearheader)
print (r.text)

In [ ]:
# shows a report-based summary, non-graphical view
url = baseurl + '/' + str(userId) + '/results'
r = requests.get(url + '/' + resultFilename, headers=bearheader)
# if the result file is not found check the error file
if (r.status_code == 404):
    r = requests.get(url + '/' + errorFilename, headers=bearheader)
print (r.text)